In [ ]:
#Laboratorio spark casos COVID 19

In [2]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1588536233611_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1588536233611_0005,pyspark,idle,Link,Link,
5,application_1588536233611_0007,pyspark,idle,Link,Link,✔


In [3]:
#import SparkSession
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#create spar session object
spark=SparkSession.builder.appName('lab_spark').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Load csv Dataset 
df=spark.read.csv('s3://amartinezvdatasets/datasets/onu/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#columns of dataframe
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [7]:
#shape of dataset
print((df.count(),len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(6211, 16)

In [8]:
#printSchema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [9]:
# Muestra las priemeras 5 filas
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado|País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|             Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-

In [14]:
from pyspark.sql.functions import udf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Funcion donde se clasifican las edades
def age_range(age):
    if age <= 14:
        return 'Niño'
    elif age >=15 and age <=20:
        return 'Adolecente'
    else:
        return 'Adulto'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Agrega la columna 'categoria edades' donde se claseifican las edades en niño, adolecente, adulto
age_udf= udf(age_range, StringType())
df.withColumn('Categoria edades', age_udf(df['Edad'])).show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+----------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |Categoria edades|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+----------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia          

In [19]:
# Función para conocer si es capital de algun mucicipio
def city(city):
    if city == 'Bogotá D.C.' or city== 'Medellín' or city == 'Cartagena de Indias':
        return 'Capital'
    else:
        return 'Municipio'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Agrega la columna 'tipo ciudadad' donde se pone si la ciudad es una capital de departamento o un municipio
city_udf= udf(city, StringType())
df.withColumn('Tipo de ciudad', city_udf(df['Ciudad de ubicación'])).show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+--------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia|FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |Tipo de ciudad|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+--------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia             |2020-02-27T00:00:00.

In [25]:
# Elimina las respectivas columnas
df_new=df.drop('Codigo DIVIPOLA', 'Tipo', 'FIS', 'fecha reporte web')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Mostrar cinco filas sin las columnas que fueron eliminadas
df_new.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+------------------------+----------+----+----+------+-------------------+---------------+-------------------+--------------------+
|ID de caso|Fecha de notificación|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|Estado|País de procedencia|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|
+----------+---------------------+-------------------+------------------------+----------+----+----+------+-------------------+---------------+-------------------+--------------------+
|         1|  2020-03-02 00:00:00|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Leve|             Italia|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|
|         2|  2020-03-06 00:00:00|Guadalajara de Buga|         Valle del Cauca|Recuperado|  34|   M|  Leve|             España|          -   -|2020-03-09 00:00:00|2020-03-19T00:00:...|
|         3|  2020-03-07 00:00:00|           Medellín|               Antioq

In [27]:
#¿Cuales son los casos importados?

df.filter(df['Tipo']=='Importado').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+---------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|     Tipo|Estado| País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+---------+------+--------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|Importado|  Leve|              Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-03-0

In [28]:
#¿Cuales son los casos importados en la ciudad de Bogotá?

df.filter((df['Tipo']=='Importado')&(df['Ciudad de ubicación'] =='Bogotá D.C.')).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+---------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|     Tipo|Estado|País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+---------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|Importado|  Leve|             Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-03-06 00

In [32]:
#¿Cuales son los casos que se han recuperado y en que ciudades?
df.filter(df['atención']=='Recuperado').select('ID de caso','Ciudad de ubicación').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+
|ID de caso|Ciudad de ubicación|
+----------+-------------------+
|         1|        Bogotá D.C.|
|         2|Guadalajara de Buga|
|         3|           Medellín|
|         4|           Medellín|
|         5|           Medellín|
|         6|             Itagüí|
|         7|Cartagena de Indias|
|         8|        Bogotá D.C.|
|         9|        Bogotá D.C.|
|        10|        Bogotá D.C.|
+----------+-------------------+
only showing top 10 rows

In [37]:
# Numero de casos por ciudad ordenados de forma descendentes
df.groupBy('Ciudad de ubicación').count().orderBy('count',ascending=False).show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Ciudad de ubicación|count|
+-------------------+-----+
|Bogotá D.C.        |2543 |
|Cali               |713  |
|Villavicencio      |344  |
|Medellín           |305  |
|Cartagena de Indias|246  |
|Barranquilla       |141  |
|Santa Marta        |139  |
|Pereira            |132  |
|Leticia            |104  |
|Neiva              |82   |
+-------------------+-----+
only showing top 10 rows

In [38]:
# Numero de casos por sexo
df.groupBy('Sexo').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Sexo|count|
+----+-----+
|   F| 2917|
|   m|    1|
|   f|    3|
|   M| 3290|
+----+-----+

In [43]:
# Numero de personas en cada estado por ciudad
df.groupBy('Estado', 'Ciudad de ubicación').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-----+
|   Estado|Ciudad de ubicación|count|
+---------+-------------------+-----+
|     Leve|           Sogamoso|    1|
| Moderado|           Sogamoso|    2|
| Moderado|       Dosquebradas|    1|
|    Grave|              Neiva|    2|
| Moderado|             Suesca|    2|
|     Leve|      Villavicencio|  334|
|     Leve|              Neiva|   73|
|     Leve|            Palmira|   42|
|     Leve|         San Andrés|    6|
|    Grave|            Ipiales|    3|
|Fallecido|            Miranda|    1|
| Moderado|             Itagüí|    1|
|     Leve|          La Dorada|   30|
|     Leve|          Contadero|    1|
|    Grave|             Ibagué|    2|
| Moderado|           El Tambo|    1|
|     Leve|           Montería|   16|
|     Leve|          Angostura|    1|
|     Leve|               Chía|   28|
|     Leve|             Calima|    1|
+---------+-------------------+-----+
only showing top 20 rows

In [50]:
# Número de fallecidos hay por ciudad

df.filter(df['Estado']=='Fallecido').groupBy('Ciudad de ubicación').count().show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Ciudad de ubicación|count|
+-------------------+-----+
|           Montería|    1|
|           El Dovio|    1|
|       Cundinamarca|    1|
|   Barranco de Loba|    1|
|              Tuluá|    1|
|              Pasto|    2|
|            Ginebra|    1|
|              Neiva|    4|
|             Ibagué|    1|
|    Barrancabermeja|    1|
|        Bucaramanga|    1|
|        Puebloviejo|    2|
|     Ciénaga de Oro|    1|
|              Bello|    1|
|          La Dorada|    2|
+-------------------+-----+
only showing top 15 rows

In [53]:
#target directory 
write_uri='s3://amartinezvdatasets/dfCovid_csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
#save the dataframe as single csv 
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…